# 1. Code Definition

In [1]:
from enum import Enum, auto
from copy import deepcopy, copy
from typing import Union, List, Optional, Iterable
from functools import reduce
from collections import Counter

In [2]:
class HashableCounter(Counter):
    def __hash__(self):
        return sum([hash((v,c)) for v,c in self.items()], 0)

    def __sub__(self, other):
        res = HashableCounter()
        res.update(super().__sub__(other))
        return res

    def __add__(self, other):
        res = HashableCounter()
        res.update(super().__add__(other))
        return res

    def __copy__(self):
        cop = HashableCounter()
        cop.update(self)
        return cop
        
    def __deepcopy__(self):
        return self.__copy__()
    
    def __eq__(self, other):
        if not isinstance(other, dict):
            return False
        
        if len(self.items()) != len(other.items()):
            return False
        
        for o,c in self.items():
            if (o not in other) or other[o] != c:
                return False
        return True

In [3]:
def is_var_negative(var, power=1):
    if isinstance(var, MulExpression):
        neg = var.negative()
    elif isinstance(var, Symbolic):
        neg = var.negative
    elif isinstance(var, Expression):
        neg = False
    else:
        neg = var < 0
    return False if (power % 2 == 0) else neg

In [4]:
class Expression:
    def __init__(self, vars: Union[List, HashableCounter]):
        self.vars = vars if isinstance(vars, HashableCounter) else HashableCounter(vars)

    def __mul__(self, other):
        if other == 0:
            return 0
        if isinstance(other, Expression):
            return sum([var*mul*other for var, mul in self.vars.items()])
        return Expression(HashableCounter({(var * other):mul for var,mul in self.vars.items()}))

    def __rmul__(self, other):
        if other == 0:
            return 0
        if isinstance(other, Expression):
            return sum([other*mul*var for var, mul in self.vars.items()])
        return Expression(HashableCounter({(other * var):mul for var,mul in self.vars.items()}))

    def __add__(self, other):
        vars = self.vars
        if isinstance(other, Expression):
            vars = self.vars + other.vars
        else:
            vars = self.vars + HashableCounter([other])
        return Expression(vars)
    
    def __radd__(self, other):
        return self.__add__(other)

    def __sub__(self, other):
        return self + (-1)*other

    def __rsub__(self, other):
        return other + (-1)*self

    def __neg__(self):
        vars = HashableCounter({-val: mul for val,mul in self.vars.items()})
        return Expression(vars)

    def __pow__(self, val):
        if val == 0:
            return 1
        if val == 1:
            return self
        base = copy(self)
        for i in range(val-1):
            base *= self
        return base

    def __str__(self):
        self_str = ""
        for var, mul in self.vars.items():
            if var == 0 or mul == 0:
                continue
            op = "-" if is_var_negative(var) else "+"
            mul_prefix = "-" if (not self_str and is_var_negative(var)) else ""
            if (mul > 0) and (mul != 1):
                mul_prefix += f"{mul}⋅"
            if (mul < 0) and (mul != -1):
                mul_prefix = mul_prefix.replace("-", "")
            var_str = mul_prefix + str(var).replace("-", "")
            self_str += f" {op} {var_str}" if self_str else var_str
        
        return self_str

    def __repr__(self):
        # I know it's wrong, but it's a QoL for notebooks
        return str(self)

    def __eq__(self, other):
        if isinstance(other, Expression):
            return self.vars == other.vars
        elif len(self.vars) == 1:
            var, mul = next(iter(self.vars.items()))
            return other == mul*var
        else:
            return False
    
    def calc(self):
        vars_cpy = self.vars.copy()
        vars_tmp = HashableCounter()
        while vars_cpy:
            var,mul = vars_cpy.popitem()
            if var == 0 or mul == 0:
                continue
            if isinstance(var, MulExpression):
                var = var.calc()
                mul *= var.extract_numeric_constants()
                neg_var = (-var).calc()
            else:
                neg_var = -var
            found = False
            found_neg = False
            for rvar in vars_tmp.keys():
                if neg_var == rvar:
                    neg_var = rvar
                    found_neg=True
                elif var == rvar:
                    mul += vars_tmp[rvar]
                    var = rvar
            if found:
                vars_tmp.pop(var)
            if found_neg:
                rmul = vars_tmp.pop(neg_var)
                if is_var_negative(var):
                    sum_mul = rmul-mul
                else:
                    sum_mul = mul-rmul
                if sum_mul > 0:
                    vars_tmp[var] = sum_mul
                elif sum_mul < 0:
                    vars_tmp[neg_var] = -sum_mul
            else:
                vars_tmp[var] = mul
                continue
        
        return Expression(vars_tmp) if vars_tmp else 0
    

    def sort_calc(self, verbose = False):
        if verbose:
            print("Expr sort", self)
        got_exp = False
        new_vars = HashableCounter()
        for var, mul in self.vars.items():
            if isinstance(var, MulExpression):
                new_var = var.sort_calc(verbose)
                if isinstance(new_var, Expression):
                    for rvar, rmul in new_var.vars.items():
                        new_vars[rvar] += mul*rmul
                        got_exp = True
                else:
                    new_vars[new_var] += mul
            else:
                new_vars[var] += mul

        new_exp = Expression(new_vars)
        return new_exp.sort_calc(verbose) if got_exp else new_exp.calc()
        

In [5]:
class Symbolic:
    def __init__(self, symbol, negative = False, imaginary = False):
        self.symbol = symbol
        self.negative = negative
        self.imaginary = imaginary
    
    def __add__(self, other):
        return Expression([self, other])

    def __radd__(self, other):
        return Expression([other, self])

    def __sub__(self, other):
        if other == self:
            return 0
        return Expression([self, -other])

    def __rsub__(self, other):
        if other == self:
            return 0
        return Expression([other, -self])
    
    def __mul__(self, other):
        if other == 0:
            return 0
        elif isinstance(other, Expression):
            return other.__rmul__(self)
        elif isinstance(other, Commutator):
            return self*other.calc()
        elif isinstance(other, MulExpression):
            return other.__rmul__(self)
        else:
            return MulExpression([self, other])

    def __rmul__(self, other):
        if other == 0:
            return 0
        elif isinstance(other, Expression):
            return other.__mul__(self)
        elif isinstance(other, Commutator):
            return other.calc()*self
        elif isinstance(other, MulExpression):
            return other.__mul__(self)
        else:
            return MulExpression([other, self])

    def __pow__(self, val):
        if val == 1:
            return self
        return MulExpression([self]*val)
    
    def _inverse(self):
            return InverseSymbolic(self.symbol, self.negative, self.imaginary)
        
    def __rtruediv__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            return (other * self._inverse()) if other > 1 else self._inverse()
        raise TypeError()

    def __neg__(self):
        return self.__class__(self.symbol, not self.negative, self.imaginary)

    def __bool__(self):
        return True

    def __eq__(self, other):
        if isinstance(other, Symbolic):
            return (type(self) == type(other)) and (self.symbol == other.symbol)
        elif isinstance(other, Expression) or isinstance(other, MulExpression):
            return other.__eq__(self)
        return False

    def __str__(self):
        return ("-" if self.negative else "") + self.symbol
    def __repr__(self):
        # I know it's wrong, but it's a QoL for notebooks
        return str(self)

    def __hash__(self):
        return hash(self.symbol) + hash(self.imaginary) + hash(self.negative)

    def __copy__(self):
        return self.__class__(self.symbol, self.negative, self.imaginary)
    

class InverseSymbolic(Symbolic):
    def __str__(self):
        return f"({'-' if self.negative else ''}1/{self.symbol})"

    def _inverse(self):
        return Symbolic(self.symbol, self.negative, self.imaginary)

    def __hash__(self):
        return hash(1) + hash(self.symbol) + hash(self.imaginary) + hash(self.negative)

In [6]:
class Constant(Symbolic):
    def _inverse(self):
        return InverseConstant(self.symbol, self.negative, self.imaginary)

class InverseConstant(InverseSymbolic, Constant):
    def _inverse(self):
        return Constant(self.symbol, self.negative, self.imaginary)

In [7]:
class MulExpression:
    def __init__(self, vars: Union[Symbolic, Iterable], constants: Optional[HashableCounter] = None):
        self.constants = constants if constants else HashableCounter()
        self.vars = []
        if isinstance(vars, Constant):
            self.constants[vars] = 1
        elif isinstance(vars, Symbolic):
            self.vars.append(vars)
        elif isinstance(vars, Iterable):
            for var in vars:
                if isinstance(var, float) or isinstance(var, int) or isinstance(var, Constant):
                    self.constants[var] += 1
                elif isinstance(var, Symbolic):
                    self.vars.append(var)
                else:
                    raise TypeError(f"Var can't be {type(vars).__name__}")
        else:
            raise TypeError(f"Vars can't be {type(vars).__name__}")

    def negative(self):
        def iterable_negative(iterable: Iterable):
            neg = False
            for var in iterable:
                if isinstance(var, Symbolic):
                    neg ^= var.negative
                else:
                    neg ^= (var < 0)
            return neg
        return iterable_negative(self.constants.elements()) ^ iterable_negative(self.vars)

    def commutable(self):
        if len(self.vars) < 2:
            return True
        for i in self.vars:
            for j in self.vars:
                if Commutator(i,j).calc() != 0:
                    return False
        return True

    def calc(self):
        simp_var_groups = []
        for group in self._get_commutable_var_groups():
            simp_vars = []
            for var in group:
                inv = 1/var
                if inv in simp_vars:
                    simp_vars.remove(inv)
                else:
                    simp_vars.append(var)
            simp_var_groups.append(simp_vars)
        simp = MulExpression(sum(simp_var_groups, []))
        num_consts = 1
        inverse = []
        for c, power in self.constants.items():
            if c in inverse:
                continue
            
            if c == 0:
                return 0

            if c == i:
                num_consts *= (-1)**int(power/2)
                if power%2 == 0:
                    continue
                else:
                    power = 1

            if isinstance(c, int) or isinstance(c, float):
                num_consts *= c**power
                continue

            if is_var_negative(c):
                c = -c
                num_consts *= -1

            inv = 1/c                
            if inv in self.constants:
                inv_pow = self.constants[inv]
                act_pow = power-inv_pow
                inverse.append(inv)
                if act_pow > 0:
                    simp.constants[c] = act_pow
                elif act_pow < 0:
                    simp.constants[inv] = -act_pow
                else:
                    continue
            elif c in simp.constants:
                simp.constants[c] += power
            else:
                simp.constants[c] = power
        if num_consts != 1:
            simp.constants[num_consts] = 1
        elif len(simp.constants) == 0 and len(simp.vars) == 0:
            return 1
        return simp
    
    def extract_numeric_constants(self) -> float:
        num = 1
        consts = list(self.constants.keys())
        for c in consts:
            if isinstance(c, int) or isinstance(c, float):
                if c < 0:
                    p = self.constants.pop(c)
                    if p % 2 == 1:
                        self.constants[-1] = 1
                        c = -c
                    num *= c**p
                else:
                    num *= (c**self.constants.pop(c))
        return num

    def __add__(self, other):
        return other.__radd__(self) if isinstance(other, Expression) else Expression([self, other])

    def __radd__(self, other):
        return other.__add__(self) if isinstance(other, Expression) else Expression([other, self])

    def __sub__(self, other):
        if other == self:
            return 0
        return other.__rsub__(self) if isinstance(other, Expression) else Expression([self, -other])

    def __rsub__(self, other):
        if other == self:
            return 0
        return other.__sub__(self) if isinstance(other, Expression) else Expression([other, -self])

    def __mul__(self, other):
        expr = copy(self)
        if isinstance(other, int) or isinstance(other, float) or isinstance(other, Constant):
            if other == 0:
                return 0
            expr.constants[other] += 1
        elif isinstance(other, Symbolic):
            expr.vars.append(other)
        elif isinstance(other, Expression):
            return other.__rmul__(self)
        elif isinstance(other, MulExpression):
            expr.constants += other.constants
            expr.vars += other.vars
        else:
            raise NotImplementedError(f"Can't multiply {self.__class__.__name__} and {type(other).__name__}")

        return expr

    def __rmul__(self, other):
        expr = copy(self)
        if isinstance(other, int) or isinstance(other, float) or isinstance(other, Constant):
            if other == 0:
                return 0
            expr.constants[other] += 1
        elif isinstance(other, Symbolic):
            expr.vars.insert(0, other)
        elif isinstance(other, Expression):
            return other.__mul__(self)
        elif isinstance(other, MulExpression):
            expr.constants += other.constants
            expr.vars = other.vars + expr.vars
        else:
            raise NotImplementedError(f"Can't multiply {self.__class__.__name__} and {type(other).__name__}")

        return expr

    def __pow__(self, val):
        if val == 1:
            return self
        base = copy(self)
        for i in range(val-1):
            base *= self
        return base
        
    def __neg__(self):
        c = HashableCounter([-1])
        c.update(self.constants)
        return MulExpression(copy(self.vars), c)
    
    def _get_commutable_var_groups(self):
        if not self.vars:
            return [[]]
        if len(self.vars) == 1:
            return [[self.vars[0]]]
        var_groups = []
        current_group = []
        prev_type = type(self.vars[0])
        for var in self.vars:
            if isinstance(var, prev_type):
                current_group.append(var)
            else:
                var_groups.append(current_group)
                current_group = [var]
                prev_type = type(var)
        var_groups.append(current_group)
        return var_groups

    def __eq__(self, other):
        if isinstance(other, MulExpression):
            if (other.constants == self.constants):
                if self.commutable() and other.commutable():
                    for var in self.vars:
                        if self.vars.count(var) != other.vars.count(var):
                            return False
                    return True
                else:
                    if self.vars and other.vars:
                        self_commutable_groups = self._get_commutable_var_groups()
                        other_commutable_groups = other._get_commutable_var_groups()
                        if len(self_commutable_groups) != len(other_commutable_groups):
                            return False
                        for i in range(len(self_commutable_groups)):
                            for var in self_commutable_groups[i]:
                                if self_commutable_groups[i].count(var) != other_commutable_groups[i].count(var):
                                    return False
                        for i in range(len(other_commutable_groups)):
                            for var in other_commutable_groups[i]:
                                if self_commutable_groups[i].count(var) != other_commutable_groups[i].count(var):
                                    return False
                        return True  # self_commutable_group is in other, and other is in self, so eq
                return False
        elif isinstance(other, Symbolic):
            if len(self.constants) == 0 and len(self.vars) == 1:
                return self.vars[0] == other
            return False
        elif isinstance(other, Expression):
            return other.__eq__(self)
        else:
            if other == 0 and any([c == 0 for c in self.constants]):
                return True
            return (not self.vars) and self.constants == tuple([other])

    def __str__(self):
        self_str = ""
        for const, power in self.constants.items():
            if const == 1 or const == -1:
                continue
            self_str += (f"{const}^{power}" if power>1 else str(const)) + "⋅"
        for grp in self._get_commutable_var_groups():
            seen = []
            for var in grp:
                if var in seen:
                    continue
                power = grp.count(var)
                self_str += (str(var) if power == 1 else (str(var) + f"^{power}")) + "⋅"
                seen.append(var)
        return ("-" if self.negative() else "") + self_str.replace("-", "")[:-1]
        
    def __repr__(self):
        # return f"{self.__class__.__name__}({list(self.constants.elements())+list(self.vars)})"
        return str(self)

    def __hash__(self):
        return sum([hash(v) for v in self.vars], hash(self.constants))

    def __copy__(self):
        return MulExpression(copy(self.vars), copy(self.constants))
    
    def sort_calc(self, verbose = False):
        if self.commutable():
            return self
        comm_groups = self._get_commutable_var_groups()
        consts = MulExpression([], self.constants)
        if len(comm_groups) == 2:
            if isinstance(comm_groups[0][0], LocationOperator):
                return self
            else:
                if verbose:
                    print("Mul sort", self)
                return consts * (MulExpression(comm_groups[1] + comm_groups[0]) - Commutator(MulExpression(comm_groups[1]), MulExpression(comm_groups[0])).calc())
        else:
            if verbose:
                print("Mul sort", self)
            i = 0
            for grp in comm_groups:
                if isinstance(grp[0], MomentumOperator):
                    break
                i += 1
            if i == (len(comm_groups) - 1):
                # sorted
                return self
            elif i == 0:
                new_exp = MulExpression(sum(comm_groups[2:], comm_groups[1] + comm_groups[0]))
                m = MulExpression(sum(comm_groups[2:], []))
                comm = Commutator(MulExpression(comm_groups[1]), MulExpression(comm_groups[0])).calc()
                return (consts*(new_exp - comm*m)).sort_calc(verbose)
            elif i == (len(comm_groups) - 2):
                new_exp = MulExpression(sum(comm_groups[:-2], []) + comm_groups[-1] + comm_groups[-2])
                m = MulExpression(sum(comm_groups[:-2], []))
                comm = Commutator(MulExpression(comm_groups[-1]), MulExpression(comm_groups[-2])).calc()
                return (consts*(new_exp - m*comm)).sort_calc(verbose)
            else:
                new_exp = MulExpression(sum(comm_groups[:i], []) + comm_groups[i+1] + comm_groups[i] + sum(comm_groups[i+2:], []))
                m1 = MulExpression(sum(comm_groups[:i], []))
                m2 = MulExpression(sum(comm_groups[i+2:], []))
                comm = Commutator(MulExpression(comm_groups[i+1]), MulExpression(comm_groups[i])).calc()
                return (consts*(new_exp - m1*comm*m2)).sort_calc(verbose)

In [8]:
class Axes(Enum):
    i = auto()
    j = auto()
    k = auto()

In [9]:
class DirectedOperator(Symbolic):
    def __init__(self, name, direction: Optional[Axes]):
        self.direction = direction
        super().__init__(name)

    def _inverse(self):
        raise NotImplemented()

    def __eq__(self, other):
        return (isinstance(other, DirectedOperator) and self.direction == other.direction) and super().__eq__(other)

    def __hash__(self):
        return hash(self.direction) + super().__hash__()

In [10]:
class LocationOperator(DirectedOperator):
   def _inverse(self):
        return InverseLocationOperator(self.symbol, self.direction)

class InverseLocationOperator(InverseSymbolic, LocationOperator):
    def _inverse(self):
        return LocationOperator(self.symbol, self.direction)    

In [11]:
class MomentumOperator(DirectedOperator):
    def _inverse(self):
        return InverseMomentumOperator(self.symbol, self.direction)

class InverseMomentumOperator(InverseSymbolic, MomentumOperator):
    def _inverse(self):
        return MomentumOperator(self.symbol, self.direction)


In [12]:
hbar = Constant("ℏ")
m = Constant("m")
omega = Constant("ω")
i = Constant("i", imaginary=True)
e = Constant("e")
m_1 = 1/m

x = LocationOperator("x", Axes.i)
y = LocationOperator("y", Axes.j)
z = LocationOperator("z", Axes.k)
r = LocationOperator("r", None)
r_1 = 1/r

px = MomentumOperator("Px", Axes.i)
py = MomentumOperator("Py", Axes.j)
pz = MomentumOperator("Pz", Axes.k)

In [13]:
class Commutator:
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def __str__(self):
        return f"[{self.left}, {self.right}]"

    def __repr__(self):
        # I know it's wrong, but it's a QoL for notebooks
        # return f"Commutator({self.left}, {self.right})"
        return str(self)

    def __copy__(self):
        return Commutator(self.left, self.right)

    def expand(self, verbose=False):
        if isinstance(self.left, Symbolic) or isinstance(self.right, Symbolic) or isinstance(self.left, Expression) or isinstance(self.right, Expression) or isinstance(self.left, MulExpression) or isinstance(self.right, MulExpression):
            expr = self._expand_commutator(verbose)
            if verbose:
                print(f"{self} -> {expr}")
            return expr
        return copy(self)

    def __add__(self, other):
        return Expression([self, other])

    def __radd__(self, other):
        return Expression([other, self])

    def __mul__(self, other):
        return self.calc()*other

    def __rmul__(self, other):
        return other*self.calc()

    def calc(self, verbose=False) -> str:
        well_known = self._calc_well_known_commutators()
        if well_known is not None:
            if verbose:
                print(f"{self} = {well_known}")
            return well_known
        else:
            expr = self.expand(verbose)
            expr = expr.calc() if isinstance(expr, Expression) or isinstance(expr, MulExpression) else expr
            if verbose:
                print(f"{self} = {expr}")
            return expr
    
    def _calc_well_known_commutators(self) -> Union[Symbolic, int]:
        if self.left == self.right:
            return 0
            
        if isinstance(self.left, Constant) or isinstance(self.right, Constant):
            # print("const")
            return 0

        if isinstance(self.left, int) or isinstance(self.left, float) or isinstance(self.right, int) or isinstance(self.right, float):
            # print("const 2")
            return 0

        if (isinstance(self.left, MulExpression) and not self.left.vars) or (isinstance(self.right, MulExpression) and not self.right.vars):
            return 0
        
        if isinstance(self.left, LocationOperator) and isinstance(self.right, LocationOperator):
            # print("locs")
            return 0

        if isinstance(self.left, MomentumOperator) and isinstance(self.right, MomentumOperator):
            # print("moment")
            return 0

        if isinstance(self.left, LocationOperator) and isinstance(self.right, MomentumOperator):
            # print("loc,mom")
            if self.left.direction is None: # means r
                if isinstance(self.left, InverseLocationOperator):
                    return -i*hbar*LocationOperator(self.right.symbol[1], self.right.direction)*(self.left**3)
                else:
                    return i*hbar*LocationOperator(self.right.symbol[1], self.right.direction)*(1/self.left)
            else:
                return 0 if self.left.direction != self.right.direction else i*hbar

        if isinstance(self.left, MomentumOperator) and isinstance(self.right, LocationOperator):
            # print("mom,loc")
            if self.right.direction is None: # means r
                if isinstance(self.right, InverseLocationOperator):
                    return i*hbar*LocationOperator(self.left.symbol[1], self.left.direction)*(self.right**3)
                else:
                    return -i*hbar*LocationOperator(self.left.symbol[1], self.left.direction)*(1/self.right)
            else:
                return 0 if self.left.direction != self.right.direction else -i*hbar
        
        return None
        
    def _expand_commutator(self, verbose=False):
        if isinstance(self.left, Expression):
            s = sum([mul*(Commutator(var, self.right).calc(verbose)) for var, mul in self.left.vars.items()])
            return s.calc() if isinstance(s, Expression) else s
        elif isinstance(self.left, MulExpression):
            consts = reduce(lambda x,y: (x**self.left.constants[x])*(y**self.left.constants[y]), self.left.constants) if self.left.constants else None
            if len(self.left.vars) > 1:
                rest_vars = MulExpression(self.left.vars[1:])
                val = self.left.vars[0]*Commutator(rest_vars, self.right).calc(verbose) + Commutator(self.left.vars[0], self.right).calc(verbose)*rest_vars
            else:
                val = Commutator(self.left.vars[0], self.right).calc(verbose)
            return consts*val if consts else val

        if isinstance(self.right, Expression):
            s = sum([mul*(Commutator(self.left, var).calc(verbose)) for var, mul in self.right.vars.items()])
            return s.calc() if isinstance(s, Expression) else s
        elif isinstance(self.right, MulExpression):
            consts = reduce(lambda x,y: (x**self.right.constants[x])*(y**self.right.constants[y]), self.right.constants) if self.right.constants else None
            if len(self.right.vars) > 1:
                rest_vars = MulExpression(self.right.vars[1:])
                val = self.right.vars[0]*Commutator(self.left, rest_vars).calc(verbose) + Commutator(self.left, self.right.vars[0]).calc(verbose)*rest_vars
            else:
                val = Commutator(self.left, self.right.vars[0]).calc(verbose)
            return consts*val if consts else val

        return (self.left*self.right - self.right*self.left).calc()

# 2. Sanity Tests

In [14]:
for op_l in (i*hbar*x, y, i):
    for op_m in (px, i*hbar*py, z):
        comm = Commutator(op_l, op_m)
        print(f"{comm} = {comm.calc()}")

# should be -h^2,0,0,0,-h^2,0,0,0,0

[i⋅ℏ⋅x, Px] = -ℏ^2
[i⋅ℏ⋅x, i⋅ℏ⋅Py] = 0
[i⋅ℏ⋅x, z] = 0
[y, Px] = 0
[y, i⋅ℏ⋅Py] = -ℏ^2
[y, z] = 0
[i, Px] = 0
[i, i⋅ℏ⋅Py] = 0
[i, z] = 0


In [15]:
Commutator(r_1**3,px).calc() # Should be -3i⋅ℏ⋅x⋅(1/r)^5

-3⋅i⋅ℏ⋅x⋅(1/r)^5

In [16]:
L = x*py - y*px
for op in (x,y,z,px,py,pz):
    comm = Commutator(L, op)
    print(f"{comm} = {comm.calc()}")
# should be ihy,-ihx,0,ihpy,-ihpx,0

[x⋅Py - y⋅Px, x] = i⋅ℏ⋅y
[x⋅Py - y⋅Px, y] = -i⋅ℏ⋅x
[x⋅Py - y⋅Px, z] = 0
[x⋅Py - y⋅Px, Px] = i⋅ℏ⋅Py
[x⋅Py - y⋅Px, Py] = -i⋅ℏ⋅Px
[x⋅Py - y⋅Px, Pz] = 0


In [17]:
Commutator(px**2, x).calc() # should be -2ihpx

-2⋅i⋅ℏ⋅Px

In [18]:
Commutator(L, px**2+py**2).calc() # should be 0

0

In [19]:
Commutator(L, x**2+y**2+z**2).calc() # should be 0

0

# 3. 2D Harmonic Oscillator

## 3.1 Operator definiton and relations

In [20]:
Hx = 0.5*m_1*px**2 + 0.5*m*(omega**2)*x**2
Hy = 0.5*m_1*py**2 + 0.5*m*(omega**2)*y**2
H = Hx + Hy
K = Hx-Hy
L = x*py - y*px
M = Commutator(K,L).calc()
print("[M,K] =", Commutator(M,K).calc())
print("[K,L] =", Commutator(K,L).calc())
print("[L,M] =", Commutator(L,M).calc())
print("[H,M] =", Commutator(H,M).calc())

[M,K] = -4.0⋅ℏ^2⋅ω^2⋅x⋅Py + 4.0⋅ℏ^2⋅ω^2⋅Px⋅y
[K,L] = -2.0⋅(1/m)⋅i⋅ℏ⋅Px⋅Py - 2.0⋅m⋅ω^2⋅i⋅ℏ⋅x⋅y
[L,M] = 2.0⋅(1/m)⋅ℏ^2⋅Py^2 - 2.0⋅ω^2⋅m⋅ℏ^2⋅x^2 - 2.0⋅(1/m)⋅ℏ^2⋅Px^2 + 2.0⋅ω^2⋅m⋅ℏ^2⋅y^2
[H,M] = 0


In [21]:
print("[M,K] == -4⋅ℏ^2⋅ω^2⋅L?", Commutator(M,K).calc() == (-4*hbar**2*omega**2*L).calc())
print("[K,L] == M?", Commutator(K,L).calc() == M)
print("[L,M] == -4⋅ℏ^2⋅K", Commutator(L,M).calc() == (-4*hbar**2*K).calc())

[M,K] == -4⋅ℏ^2⋅ω^2⋅L? True
[K,L] == M? True
[L,M] == -4⋅ℏ^2⋅K True


## 3.2 Normalization

In [22]:

A1 = (0.25*(1/hbar)*(1/i)*(1/omega)*M).calc()
A2 = (0.5*(1/omega)*K).calc()
A3 = (0.5*L).calc()
print("A1 =", A1)
print("A2 =", A2)
print("A3 =", A3)

A1 = -0.5⋅ω⋅m⋅x⋅y - 0.5⋅(1/ω)⋅(1/m)⋅Px⋅Py
A2 = -0.25⋅ω⋅m⋅y^2 - 0.25⋅(1/ω)⋅(1/m)⋅Py^2 + 0.25⋅ω⋅m⋅x^2 + 0.25⋅(1/ω)⋅(1/m)⋅Px^2
A3 = -0.5⋅y⋅Px + 0.5⋅x⋅Py


In [23]:
print("[A1, A2] == i⋅ℏ⋅A3?", Commutator(A1,A2).calc() == (i*hbar*A3).calc())
print("[A2, A3] == i⋅ℏ⋅A1?", Commutator(A2,A3).calc() == (i*hbar*A1).calc())
print("[A3, A1] == i⋅ℏ⋅A2?", Commutator(A3,A1).calc() == (i*hbar*A2).calc())

[A1, A2] == i⋅ℏ⋅A3? True
[A2, A3] == i⋅ℏ⋅A1? True
[A3, A1] == i⋅ℏ⋅A2? True


In [24]:
expr = (A1**2+A2**2+A3**2).calc()
(expr.calc()-(0.25*(1/omega)**2*H**2).calc()).calc().sort_calc()

-0.25⋅ℏ^2 - 0.25⋅y^2⋅Px^2 - 0.25⋅Py^2⋅x^2

$A^2-\dfrac{1}{(4\omega^2)}H^2=-\dfrac{1}{4}\hbar^2 ==> A^2 = \dfrac{1}{(4\omega^2)}H^2 - \dfrac{1}{4}\hbar^2$

# 4. 3D Hydrogen Atom

### 4.1 Operator definitons

In [25]:
Lx = y*pz-z*py
Ly = z*px-x*pz
Lz = x*py-y*px

Mx = 0.5*m_1*(py*Lz-pz*Ly-Ly*pz+Lz*py)-x*r_1*e**2
My = -0.5*m_1*(px*Lz-pz*Lx-Lx*pz+Lz*px)-y*r_1*e**2
Mz = 0.5*m_1*(px*Ly-py*Lx-Lx*py+Ly*px)-z*r_1*e**2

### 4.2 Commutators

In [26]:
print("[Px, Lx] =", Commutator(px, Lx).calc())
print("[Px, Ly] =", Commutator(px, Ly).calc())
print("[Px, Lz] =", Commutator(px, Lz).calc())

print("[Py, Lx] =", Commutator(py, Lx).calc())
print("[Py, Ly] =", Commutator(py, Ly).calc())
print("[Py, Lz] =", Commutator(py, Lz).calc())

print("[Pz, Lx] =", Commutator(pz, Lx).calc())
print("[Pz, Ly] =", Commutator(pz, Ly).calc())
print("[Pz, Lz] =", Commutator(pz, Lz).calc())

[Px, Lx] = 0
[Px, Ly] = i⋅ℏ⋅Pz
[Px, Lz] = -i⋅ℏ⋅Py
[Py, Lx] = -i⋅ℏ⋅Pz
[Py, Ly] = 0
[Py, Lz] = i⋅ℏ⋅Px
[Pz, Lx] = i⋅ℏ⋅Py
[Pz, Ly] = -i⋅ℏ⋅Px
[Pz, Lz] = 0


In [27]:
print("[Lx, Ly] == i⋅ℏ⋅Lz?", Commutator(Lx, Ly).calc() == (i*hbar*Lz).calc())
print("[Ly, Lz] == i⋅ℏ⋅Lx?", Commutator(Ly, Lz).calc() == (i*hbar*Lx).calc())
print("[Lz, Lx] == i⋅ℏ⋅Ly?", Commutator(Lz, Lx).calc() == (i*hbar*Ly).calc())

[Lx, Ly] == i⋅ℏ⋅Lz? True
[Ly, Lz] == i⋅ℏ⋅Lx? True
[Lz, Lx] == i⋅ℏ⋅Ly? True


In [28]:
print("[Mx, Lx] =", Commutator(Mx, Lx).calc())
print("[My, Ly] =", Commutator(My, Ly).calc())
print("[Mz, Lz] =", Commutator(Mz, Lz).calc())

[Mx, Lx] = 0
[My, Ly] = 0
[Mz, Lz] = 0
